<a href="https://colab.research.google.com/github/id-shiv/project_notebooks/blob/master/%5BProject_102%5D_Chatbots_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intents

In [0]:
data = {
    "intents": [{
        "tag": "greeting",
        "patterns": [
                     "Hello", 
                     "Hi",
                     "Howdy",
                     "Whats up"
        ],
        "responses": [
                      "Hello!",
                      "Good to see you!, how may i help you",
                      "Hi! how may i help you"
        ],  
        "context_set": ""
    },
    {
        "tag": "name",
        "patterns": [
                     "Whats your name?", 
                     "your name",
                     "who are you"
        ],
        "responses": [
                      "My name! Anonymous",
                      "I am Anonymous"
        ],  
        "context_set": ""
    }
    ]
}

# Chatbot

## Libraries

In [60]:
import nltk
nltk.download('punkt')

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow as tf
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data

In [55]:
print(data['intents'])

[{'tag': 'greeting', 'patterns': ['Hello', 'Hi', 'Howdy', 'Whats up'], 'responses': ['Hello!', 'Good to see you!, how may i help you', 'Hi! how may i help you'], 'context_set': ''}, {'tag': 'name', 'patterns': ['Whats your name?', 'your name', 'who are you'], 'responses': ['My name! Anonymous', 'I am Anonymous'], 'context_set': ''}]


### Prepare

Get words, labels and patterns vs intents

In [56]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    word_s = nltk.word_tokenize(pattern)
    words.extend(word_s)
    
    docs_x.append(pattern)
    docs_y.append(intent['tag'])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])

print(words)
print(labels)
print(docs_x)
print(docs_y)

['Hello', 'Hi', 'Howdy', 'Whats', 'up', 'Whats', 'your', 'name', '?', 'your', 'name', 'who', 'are', 'you']
['greeting', 'name']
['Hello', 'Hi', 'Howdy', 'Whats up', 'Whats your name?', 'your name', 'who are you']
['greeting', 'greeting', 'greeting', 'greeting', 'name', 'name', 'name']


Get vocabulary

In [57]:
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))
print(words)

labels = sorted(labels)
print(labels)

['?', 'ar', 'hello', 'hi', 'howdy', 'nam', 'up', 'what', 'who', 'yo', 'you']
['greeting', 'name']


Bag of words (one hot encoding)

In [0]:
def one_hot_encoder(document, words):
  bag = []
  document_words = [stemmer.stem(word) for word in nltk.word_tokenize(document)]

  for word in words:
    if word in document_words:
      bag.append(1)
    else:
      bag.append(0)
  
  return numpy.array(bag)

Prepare features and labels

In [59]:
training = []
output = []

out_empty = [0 for _ in range (len(labels))]

for x, doc in enumerate(docs_x):
  bag = one_hot_encoder(doc, words)
  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

print(training)
print(output)

[[0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 1 0 1 0 1 0]
 [0 0 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 1 0 1]]
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]]


### View

In [63]:
print(f'Number of words in vocabulary (features) : {len(words)}')
print(f'Number of labels : {len(labels)}')
print(f'Number of samples : {len(training)}')

Number of words in vocabulary (features) : 11
Number of labels : 2
Number of samples : 7


## Model

In [0]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(words)])
net = tflearn.fully_connected(net, 8)  # hidden layer with 8 neurons
net = tflearn.fully_connected(net, 8)

#  Output layer
net = tflearn.fully_connected(net, len(labels), activation='softmax')

net = tflearn.regression(net)

model = tflearn.DNN(net)

### Train

In [0]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save('chatbot.tflearn')

## Predict

In [68]:
model.load('chatbot.tflearn')

INFO:tensorflow:Restoring parameters from /content/chatbot.tflearn


In [0]:
def chat():
  print('Bot is Ready!, type quit to exit ...')
  while True:
    text = input('You : ')
    if text.lower() == "quit":
      print('See you again! Bye')
      break
    
    # Predict
    encoded = one_hot_encoder(text, words)
    prediction = model.predict([encoded])

    predicted_label_index = numpy.argmax(prediction)
    predicted_label = labels[predicted_label_index]
    
    # Respond
    for intent in data['intents']:
      if intent['tag'] == predicted_label:
        responses = intent['responses']
    print(random.choice(responses)) 


In [80]:
chat()

Bot is Ready!, type quit to exit ...
You : Hi
Good to see you!, how may i help you
You : what is your name , do you have one?
My name! Anonymous
You : great, thanks of help
Hello!
You : quit
See you again! Bye
